In [1]:
import pandas as pd
import numpy as np

In [2]:
#import excel spread sheet of list of violations
violations_list = pd.read_excel("../data/Codes' List of Violations 15 aug 2017_edit.xlsx")

In [3]:
#check out the columns to see how they were importated and how NULL values are treated
violations_list.head(1)

,VIOLATION_ID,ORG_ID,VIOLATION_TYPE,VIOLATION_DESC,VIOLATION_TEXT,REMEDIAL_TEXT,TABLE_NAME,EXPIRED_FLAG,DATE_EXPIRED,CREATED_BY,DATE_CREATED,MODIFIED_BY,DATE_MODIFIED,FEE_SETUP_ID,Unnamed: 14
0,1,1,BANNERS,Banners,Section 17.32.060 & 17.32.070 - Banners: Bann...,CAAZ_BANNERS,NaT,N,NaN,1.0,2014-07-09 08:03:52.640000,NaN,NaT,NaN,NaN


In [4]:
#check how complete each column is. VIOLATION_DESC seems to be most complete with only 9 null objects (compared to range index)
violations_list.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 934 entries, 0 to 933
Data columns (total 15 columns):
VIOLATION_ID      928 non-null object
ORG_ID            789 non-null object
VIOLATION_TYPE    788 non-null object
VIOLATION_DESC    925 non-null object
VIOLATION_TEXT    775 non-null object
REMEDIAL_TEXT     345 non-null object
TABLE_NAME        137 non-null datetime64[ns]
EXPIRED_FLAG      651 non-null object
DATE_EXPIRED      2 non-null object
CREATED_BY        650 non-null float64
DATE_CREATED      651 non-null object
MODIFIED_BY       23 non-null object
DATE_MODIFIED     22 non-null datetime64[ns]
FEE_SETUP_ID      0 non-null float64
Unnamed: 14       0 non-null float64
dtypes: datetime64[ns](2), float64(3), object(10)
memory usage: 109.5+ KB


In [5]:
#drop columns: ORG_ID, TABLE_NAME, EXPIRED_FLAG, DATE_EXPIRED, CREATED_BY, DATE_CREATED, MODIFIED_BY, DATE_MODIFIED, FEE_SETUP_ID, Unnamed:14
violations_trimmed = violations_list.drop(["ORG_ID", "TABLE_NAME", "EXPIRED_FLAG", "DATE_EXPIRED", "CREATED_BY", "DATE_CREATED", "MODIFIED_BY", "DATE_MODIFIED", "FEE_SETUP_ID", "Unnamed: 14"], axis=1)

In [6]:
#check to make sure columns were dropped as expected.
violations_trimmed.head(1)

,VIOLATION_ID,VIOLATION_TYPE,VIOLATION_DESC,VIOLATION_TEXT,REMEDIAL_TEXT
0,1,BANNERS,Banners,Section 17.32.060 & 17.32.070 - Banners: Bann...,CAAZ_BANNERS


In [7]:
violations_trimmed.columns = ["id", "type", "desc", "text", "remedial"]
violations_renamed = violations_trimmed
violations_renamed.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 934 entries, 0 to 933
Data columns (total 5 columns):
id          928 non-null object
type        788 non-null object
desc        925 non-null object
text        775 non-null object
remedial    345 non-null object
dtypes: object(5)
memory usage: 36.6+ KB


In [8]:
#these commands subset data into smaller groups with common traits in the text column to interrogate each subset independently.
#also remove entries with NaN entries to clean df
#identifies rows with information entry errors (descriptions in id column)

#subset that contains rows with the regular expression "Section #." in text column
violations_section = violations_renamed[violations_renamed.text.str.match(r"(S|s)ection\s(\d+)\.", na=False)]

#new df where rows containing "Section #" are removed
violations_nosection = violations_renamed.drop(violations_section.index)

#subset that contains rows with variations of TCA in text column
violations_TCA = violations_nosection[violations_nosection.text.str.match("(T.C.A|Tennessee Code Annotated)", na=False)]

#new df where rows containing "T.C.A" or Tennessee Code Annotated" are removed
violations_noTCA = violations_nosection.drop(violations_TCA.index)

#subset that contains variations of MCL in text column 
#Be aware MCL and MCL Chapter appear to be different types of codes
violations_MCL = violations_noTCA[violations_noTCA.text.str.match("(M.C.L|MCL Chapter)", na=False)]

#new df where rows containing "M.C.L" or "MCL Chapter" are removed
violations_noMCL = violations_noTCA.drop(violations_MCL.index)
#violations_noMCL

In [9]:
#subset of rows containing "-" in id column. These are errors.
violations_dash = violations_noMCL[violations_noMCL['id'].str.match("-", na=False)]
#violations_dash

#new df with rows containing "-" in id column removed
violations_dash = violations_noMCL.drop(violations_dash.index)

In [10]:
#subset of rows containing strings in id column. These values may be related to other entries or misplaced.
violations_description_id = violations_noMCL[violations_noMCL['id'].str.match("(\(|[a-zA-Z]|[0-9.])", na=False)]
#violations_description_id

#new df with descriptions in id column removed
violations_description_removed = violations_dash.drop(violations_description_id.index)
#violations_description_removed

#subset of rows containing NaN in type column
violations_NaN_type = violations_description_removed[violations_description_removed['type'].isnull()]

#new df where entries with NaN values in type column are removed
violations_remainder = violations_description_removed.drop(violations_description_removed[violations_description_removed['type'].isnull()].index)
violations_remainder

,id,type,desc,text,remedial
126,127,MVENVEND,Mobile Vending,Ordinance BL2006-1283 - Mobile Vendor: The ped...,CAAZ_MVEN_VENDING
196,197,TRASHCANS,Trash Cans,Ordinance 89-826 - Trash Cans: Refuse containe...,CAAH_TRASH_CANS
209,210,SWVIOLPTP,Violations - Post Construction Treatment,See SW Code,NaN
210,212,SWVIOLWQ,Violations for Water Quality Impacts,See SW Code,NaN
211,213,SWENVCRT,Environmental Court Proceedings,See SW Code,NaN
212,214,SWVIOLGP,Violations - for Construction Activities,See SW Code,NaN
213,247,917,HOUSE DISREPAIR,NaN,NaN
215,384,CAAA_BOAT,BOAT STORAGE IN FRONT YARD PROHIBITED,No trailer or watercraft shall be stored in th...,NaN
226,395,CAAA_LANDLORD_REGSTR,LANDLORD REGISTRATION,Failure to register as a landlord. T.C.A. 66-...,NaN
236,405,CAAA_TRASH_CANS,TRASH CANS,Ord. 89-826 - Trash Cans: Refuse containers sh...,NaN


In [11]:
#export csv files containing subsets for section, TCA, MCL, descriptions in id column, and remaining entries.
violations_section.to_csv("../data/violations_section.csv")
violations_TCA.to_csv("../data/violations_TCA.csv")
violations_MCL.to_csv("../data/violations_MCL.csv")
violations_description_id.to_csv("../data/violations_description_id.csv")
violations_remainder.to_csv("../data/violations_remainder.csv")